# Downloading Historical Weather Webpage from Weather Underground

The Goal of the following section is to download the appropriate webpages that contain the historical weather data.  It was originally written by someone at 538 and I downloaded it as a way to learn how to scrape the web for data.  

- It downloads the intire html webpage in the next code block
- then opens it
- looks for the data you want on the webpage using tags
- Finally it writes that data to a file in the next cell as a csv file for later use.

This is my first data science individual project, so hopefully I dont royally screw it up!


The end goal is to have a csv file that has the hourly historical weather data from weather underground for a given range of dates.  It should include the date and time of the data as well as all of the columns that weather underground has for each hour.

enjoy!


# 1) Scraping the Web for the Webpages with Weather Data in them
## 1a) import the libraries we will need, and make date variables

Start by making some functions that will be used for downloading so that the code we write later on will be less of a big block of text, and more of simple names of functions that are called.


In [35]:
from datetime import datetime, timedelta
from urllib.request import urlopen
import os
import time
import numpy as np


#define the list of stations to look at
stations = ['KCMI',
           'KBOS']

#set up starting date
starting_year = 2017
starting_month = 10
starting_day = 31

#set up ending date
ending_year = 2017
ending_month = 11
ending_day = 4



#putting the dates into variales that can be used
start_date = datetime(year=starting_year, month = starting_month, day = starting_day)
end_date = datetime(year=ending_year,month=ending_month, day=ending_day)

current_date = start_date

## 1b) Func: Formating the URL given a station and date


The first thing I want to create a function that  will format and create the string that is the URL to download

The main purpose of splitting this up into more and more cells is so that it is easier for someone else to read, or for me to remember what I was thinking when I was doing.  It also works well as a rubber duck dbugging method for me.  

In [36]:
def wundgnd_hourly_URLgen (station, current_date):

    '''
    the input for this will be a time object with the year, month, day and a time of all zeros.  
    Having the time be all zeros will be important for generating time stamps for each row of data from the table later on

    the output will be a string that is the URL of the weather underground page that we want to have downlaoded
    '''
    #input the base URL that we will use to get the data
    lookup_URL = 'http://www.wunderground.com/history/airport/{}/{}/{}/{}/DailyHistory.html'
        
    #generate the url for this location and date
    formatted_lookup_URL = lookup_URL.format(station,
                    current_date.year,current_date.month,current_date.day)
    return formatted_lookup_URL
    
#URL_to_write = wundgnd_hourly_URLgen('CMI', start_date )
#print(URL_to_write)

Now that looks like what we want it to do.  This will only work for the specific URL that we are dealing with.  
to scrape a different webpage (that is still just time formated) all you need to do is figure out where the
date/location/other variable needs to go and format it in that way.  

##  1c) Func: Take in a URL and save the webpage

the nest step in the process is to take the URL that was just generated, find the page, and download it.  
Lets write a function to do this so that the actual running of the code is less brain intensive for us.

This will be for a specific date, and a specific airport because that is how the webpages are organized

In [37]:

URL_to_write = wundgnd_hourly_URLgen('CMI', start_date )

def wundgnd_hourly_pagesave (formatted_lookup_URL,station, current_date):
    '''  The goal of this function is to take in a URL and save the file to a folder named with the station abreviation
    
    the input should be a string that is a valid URL, a station name, and a date object called current_date
    
    there will be no formal output, but the result should be an html file saved in the folder we want it saved into
    
    BE CAREFUL
    The open as writing will just overwrite any file you have in there.  This means that every time it will overwrite
    whatever you have in there with that file name.
    
    '''
    
    #pull the html file in that is at the url that we want.
    html = urlopen(formatted_lookup_URL).read().decode('utf-8')

    #generate the file name for the url to be saved
    out_file_name = '{}/html_files/{}-{}-{}.html'.format(station, current_date.year,
                                                  current_date.month,
                                                  current_date.day)
    #open the file just generated to write, which overwrites any old files you might have had.
    with open(out_file_name, 'w') as out_file:
        out_file.write(html)
        
    #print('Wrote file',out_file_name )
    
#testing the function
#wundgnd_hourly_pagesave(URL_to_write,'CMI',current_date)

## 1d) Func: Downloading all the Webpages we Need

now that we have defined the functions for generating the URL and downloading the page,
the next step is to iterate through the dates and stations that we want to download info for

It should be much shorter than the older version

In [38]:
def scrape_station(station, start_date, end_date):
    '''
    This function scrapes the weather data web pages from wunderground.com
    for the station you provide it in the date range provided.

    You can look up your city's weather station by performing a search for
    it on wunderground.com then clicking on the "History" section.
    The 4-letter name of the station will appear on that page.
    '''

    #initialize the loop at the variable at the starting day
    current_date = start_date

    # See if a directory exists in the local folder, and if it doesnt, make one
    saving_folder = station +'/html_files/'
    #print(saving_folder)
    if os.path.isdir(station) != True :
        os.mkdir(station)
        if os.path.isdir(saving_folder) != True:
            os.mkdir(saving_folder)
            
    print(saving_folder)
    #Iterate through the days to download each webpage
    while current_date != end_date:

        print('\n(',datetime.now().strftime('%Y-%m-%d %H:%M:%S'),') Working on',station, 'data from ', current_date)

        #generate the URL to look up
        URL_to_write = wundgnd_hourly_URLgen (station, current_date)

        #generate the file name for the url to be saved
        wundgnd_hourly_pagesave(URL_to_write,station,current_date)
        
        #wait a second and then iterate the day.  Dont want to get blocked!
        time.sleep(1) 
        current_date += timedelta(days=1)


# Run the function to scrape the station we are interested in
for element in stations:
    #print(element)
    scrape_station(element, start_date, end_date)



KCMI/html_files/

( 2017-11-05 21:10:10 ) Working on KCMI data from  2017-10-31 00:00:00

( 2017-11-05 21:10:11 ) Working on KCMI data from  2017-11-01 00:00:00

( 2017-11-05 21:10:13 ) Working on KCMI data from  2017-11-02 00:00:00

( 2017-11-05 21:10:15 ) Working on KCMI data from  2017-11-03 00:00:00
KBOS/html_files/

( 2017-11-05 21:10:17 ) Working on KBOS data from  2017-10-31 00:00:00

( 2017-11-05 21:10:18 ) Working on KBOS data from  2017-11-01 00:00:00

( 2017-11-05 21:10:20 ) Working on KBOS data from  2017-11-02 00:00:00

( 2017-11-05 21:10:22 ) Working on KBOS data from  2017-11-03 00:00:00


Great!  Now we have each webpage downlaoded and the code is easy to read and understand.  

Now would be a good time to check to see that they downloaded to a place that you think they should have downloaded to.  

# 2) Pulling the Data off of the Downloaded Webpages

## 2a) Func: Pull in the webpage and output a data structure with all the data in it

For the next step, after downloading all of the webpages to the local server, we want to pull the data off of the html formatted wepage.   This will require going through each of the html files and 

In [39]:

def wundgnd_hourly_tableextract (file_in_name):

    '''
    The goal of this function is to intake a file name (that presumably exists) and output the html from the table
    that we are actually interested in (the one with the historical hourly weather data in it)
    
    input: File name as a string
    
    output: The raw table html without formatting or extracting data
    
    '''
    from datetime import datetime, timedelta
    from bs4 import BeautifulSoup
    from urllib.request import urlopen
    
    with open(file_in_name) as in_file:
                
        #extract the html using Beautiful Soup into a variable called soup
        soup = BeautifulSoup(in_file.read(), 'html.parser')
        
        #print(soup)
        #Find the table, and define the table_html variable so we can use it
        page_html =  soup.findAll("div", {"class":"high-res"})
        #print(page_html)
        
        table_html = page_html[0]
        #print(table_html)        
        data_array = []
        #in the table that is html formatted, find each row via its class and its label
        row_html = table_html.findAll('tr',{"class" : "no-metars"})
        #print(row_html)
        for row in row_html:
            
            #print(row.text.split())
            data_array.append(row.text.split())
            
        #print(data_array)    
        
        return data_array
    
file_in_name = '{}/html_files/{}-{}-{}.html'.format(stations[0], current_date.year,
                                                  current_date.month,
                                                  current_date.day)
data = wundgnd_hourly_tableextract(file_in_name)

#print(data[0])

We now have the output that is a list of lists of strings of each of the rows of the table from the page.  Upward and Onward!  

In [40]:
print(len(data))

#for i in range(len(data)):
    #print(data[i])


23


### 2c) Time Time Time, time to figure it out

Once we get the data imported, we want to be able to note what time each of the rows are from.  Since the time 
from the data will be a string, lets write a function that will change from hh:mm am/pm to a time object we can add to the date and change the time stamp for that data column.  

In [41]:
def hhmm2time (string_hh_mm, ampm = 'am'):
    
    """ the goal of this is to make a program that intakes a time that is a string and can ouput
    a time in hours and minutes that can be added to the time variables.  
    
    The best way to do that might be tomake something that is a time difference in of itself.  
    Ill start working on doign that.  
    
    The imput should be a time of the format 'hh:mm'
        - it does not matter if there is 1 or 2 hour or minute digits
        - The defaul is to have the time in 24 hour time rather than 12 hour
        - If the time is in 12 hour, include am or pm as the second input and it will adjust for it
    
    The output will be a time delta object that can be added to the curren time to have
        the correct time stamp in the data
    
    """
    
    #split the time up into hours and minutes
    [hours , minutes] = string_hh_mm.split(':')
    
    #adjust for 12 hr vs 24 hour time and define the output variable
    if ampm == 'pm': 
        output = timedelta(hours = (int(hours)+12), minutes = int(minutes))
    else:
        output = timedelta(hours = (int(hours)), minutes = int(minutes))
    
    return output

#testing the function
string_hh_mm = '9:53'

print(string_hh_mm)
print(current_date)

print(current_date + hhmm2time(string_hh_mm,'pm'))


9:53
2017-10-31 00:00:00
2017-10-31 21:53:00


# 2b) Cleaning and Formating the Data We want from the Table

So this is the real nitty gritty part of data science, where you have imported the data, its in some format, and we need to figure out how to make sense of the whole thing.  For this data set the biggest porblem is that the second column is wind chill, which is not listesd for every every day, and not even neccearily listed for every hour.  Also the fact that the units on each of the numbers is broken out into another column (which could be useful...)

So here goes, to start, I will just make 3 data pieces, and pull them off so that I can plot them.  

- Time (as hh:mm)
- Time of Day (as am or pm)
- Temperature (as dd.d)
- What ever the place after temperature is


Luckey for me, the main thing I am looking for (temperature vs time) are the first two columns, and they should work without a problem.  

In [42]:
#print(data[0])

def wundgnd_hourly_datacleaning (current_date, data):
    '''
    The goal of this function is to take the string of srings that wundgnd_hourly_tableextract outputs 
    (or whatever other data structure that eventually gets changed to) and make variables for each 
    value that we are interested in (right now just time, am/pm, temperature, the next thing after temp)
    
    the input : a list of a list of strings or some other data structure (pandas data fram eventually?)
    
    output : list of lists that is a regular shape (square) and has the values we want in it
            Another output should be the header file that we want the file to have.  
    
    
    '''
    
    import numpy as np
    import math
    #print(data[0])
    
    
    #creating the header for the data file
    
    header = ['Time Stamp', 'AirTemp(F)','AirTemp(c)']
    units = ['yyyy-mm-dd hh:mm:ss', 'F', 'C']
    data_cleaned = []
    
    for row in data[0:]:
        
        #print(row)
        #row is a list of strings that is a row of the weather data table.  Aka all data for a given time

        #define each of the data variables that we want to output
        data_hhmm_str = row[0]
        data_ampm = row[1]
        date_and_time = str(current_date + hhmm2time(data_hhmm_str,data_ampm))
                
        data_temp_f = float(row[2])
        data_temp_c = math.ceil((10.*(float(data_temp_f)-32)*5/9))/10
        
        #print(data_time , data_ampm , data_temp_f, data_temp_c)
        
        data_list = [date_and_time, data_temp_f, data_temp_c]
        #print(data_list)
        
        # data_str = 
        data_cleaned.append(data_list)
    
    return [header, units, data_cleaned]
[data_header, data_units, data_list] = wundgnd_hourly_datacleaning (current_date, data)

print(data_header)
print(data_units)
print(data_list)

['Time Stamp', 'AirTemp(F)', 'AirTemp(c)']
['yyyy-mm-dd hh:mm:ss', 'F', 'C']
[['2017-10-31 12:53:00', 35.1, 1.8], ['2017-10-31 01:53:00', 37.9, 3.3], ['2017-10-31 02:53:00', 37.9, 3.3], ['2017-10-31 03:53:00', 37.0, 2.8], ['2017-10-31 04:53:00', 37.0, 2.8], ['2017-10-31 05:53:00', 35.1, 1.8], ['2017-10-31 06:53:00', 30.9, -0.6], ['2017-10-31 07:53:00', 30.0, -1.1], ['2017-10-31 08:53:00', 35.1, 1.8], ['2017-10-31 09:53:00', 37.9, 3.3], ['2017-10-31 10:53:00', 39.0, 3.9], ['2017-10-31 11:53:00', 39.9, 4.4], ['2017-10-31 12:53:00', 42.1, 5.7], ['2017-10-31 01:53:00', 41.0, 5.0], ['2017-10-31 02:53:00', 42.1, 5.7], ['2017-10-31 03:53:00', 43.0, 6.2], ['2017-10-31 04:53:00', 42.1, 5.7], ['2017-10-31 05:53:00', 39.9, 4.4], ['2017-10-31 06:53:00', 39.0, 3.9], ['2017-10-31 07:53:00', 37.9, 3.3], ['2017-10-31 09:53:00', 37.0, 2.8], ['2017-10-31 10:53:00', 37.0, 2.8], ['2017-10-31 11:53:00', 37.9, 3.3]]


so that is a wrap on the wundgnd_hourly_datacleaning function that takes in a list of lists, and pulls out the values we want, creates a header (that is defined in the function) and outputs a list of lists (or some other data structure) that we can use to manipulate the data and save it to a csv file.  

Nice Work Team!

In [43]:
#First lest see what the data looks like.  

print(data_list)
print(station)

[['2017-10-31 12:53:00', 35.1, 1.8], ['2017-10-31 01:53:00', 37.9, 3.3], ['2017-10-31 02:53:00', 37.9, 3.3], ['2017-10-31 03:53:00', 37.0, 2.8], ['2017-10-31 04:53:00', 37.0, 2.8], ['2017-10-31 05:53:00', 35.1, 1.8], ['2017-10-31 06:53:00', 30.9, -0.6], ['2017-10-31 07:53:00', 30.0, -1.1], ['2017-10-31 08:53:00', 35.1, 1.8], ['2017-10-31 09:53:00', 37.9, 3.3], ['2017-10-31 10:53:00', 39.0, 3.9], ['2017-10-31 11:53:00', 39.9, 4.4], ['2017-10-31 12:53:00', 42.1, 5.7], ['2017-10-31 01:53:00', 41.0, 5.0], ['2017-10-31 02:53:00', 42.1, 5.7], ['2017-10-31 03:53:00', 43.0, 6.2], ['2017-10-31 04:53:00', 42.1, 5.7], ['2017-10-31 05:53:00', 39.9, 4.4], ['2017-10-31 06:53:00', 39.0, 3.9], ['2017-10-31 07:53:00', 37.9, 3.3], ['2017-10-31 09:53:00', 37.0, 2.8], ['2017-10-31 10:53:00', 37.0, 2.8], ['2017-10-31 11:53:00', 37.9, 3.3]]
['KCMI', 'KBOS']


# 3) Looping Through days and Saving the Data to a file

## 3a) taking in the data, and saving it to a file

Ok, so we have the data that we are interested in, and we have dates and times, and temperatures.  Now its time to either do analyiss on it, or save it to a file.  Lets work on making a function that saves the data to a file.  

## 3b) Putting it all together

Now that we have all the pieces in place, we need to write a for loop that will iterate through all of the dates and locations.



In [51]:
for station in stations:
    print('Working on ',station)
    current_date = start_date
    
    with open('{}/weather_hourly_{}_{}-{}-{}_to_{}-{}-{}.csv'.format(station,station,current_date.year,
                                                             current_date.month,current_date.day,end_date.year,
                                                             end_date.month,end_date.day), 'w') as out_file:
        
        #convert the header to a comma separated string so it can be written to the file.
        header_to_write = ",".join (map(str,data_header))+'\n'
        out_file.write(header_to_write)
        
        units_to_write = ",".join(map(str,data_units))+'\n'
        out_file.write(units_to_write)  
            
            
          
        while current_date != end_date:
            #First pull in the data off of the webpages that have already been downloaded
            print(current_date)
            
            file_in_name = '{}/html_files/{}-{}-{}.html'.format(stations[0], current_date.year,
                                                  current_date.month,
                                                  current_date.day)
            
            data = wundgnd_hourly_tableextract(file_in_name)
        
            #make the data strucutres that you want with the data cleaning function
            [data_header, data_units, data_list] = wundgnd_hourly_datacleaning (current_date, data)
            
              
            #opening the webpage that the information is in according to the labels from the other function
            with open('{}/html_files/{}-{}-{}.html'.format(station, current_date.year, 
                                                current_date.month, current_date.day)) as in_file:
                for row in data_list:
                    data_to_write =  ",".join(map(str,row))+'\n'
                    #print(data_to_write)
                    out_file.write(data_to_write)
                    #print(data_to_write)
    
         
            #iterate the day 1 day
            current_date += timedelta(days=1)
        

Working on  KCMI
2017-10-31 00:00:00
2017-11-01 00:00:00
2017-11-02 00:00:00
2017-11-03 00:00:00
Working on  KBOS
2017-10-31 00:00:00
2017-11-01 00:00:00
2017-11-02 00:00:00
2017-11-03 00:00:00
